This is an [R Markdown](http://rmarkdown.rstudio.com) Notebook. When you execute code within the notebook, the results appear beneath the code. 

Try executing this chunk by clicking the *Run* button within the chunk or by placing your cursor inside it and pressing *Ctrl+Shift+Enter*. 

# Load some useful functions

In [ ]:
library(pagoda2)
library(Seurat)
library(ggthemes)
sn <- function(x) { names(x) <- x; return(x); }
# load 10x matrices from a named list of result folders
t.load.10x.data <- function(matrixPaths) {
  require(parallel)
  require(Matrix)
  mclapply(sn(names(matrixPaths)),function(nam) {
    matrixPath <- matrixPaths[nam];
    # read all count files (*_unique.counts) under a given path
    #cat("loading data from ",matrixPath, " ");
    x <- as(readMM(gzfile(paste(matrixPath,'matrix.mtx.gz',sep='/'))),'dgCMatrix'); # convert to the required sparse matrix representation
    cat(".")
    gs <- read.delim(gzfile(paste(matrixPath,'features.tsv.gz',sep='/')),header=F)
    rownames(x) <- gs[,2]
    cat(".")
    gs <- read.delim(gzfile(paste(matrixPath,'barcodes.tsv.gz',sep='/')),header=F)
    colnames(x) <- gs[,1]
    cat(".")
    colnames(x) <- paste(nam,colnames(x),sep='_');
    x
  },mc.cores=30)
}

doUMAP <- function(PCA,n_neighbors,min_dist,max_dim=2,seed.use=42){
  require(reticulate)
  if (!is.null(x = seed.use)) {
    set.seed(seed = seed.use)
    py_set_seed(seed = seed.use)
  }
  umap_import <- import(module = "umap", delay_load = TRUE)
  umap <- umap_import$UMAP(n_neighbors = as.integer(x = n_neighbors), 
                           n_components = as.integer(x = max_dim), metric = "correlation", 
                           min_dist = min_dist)
  
  umap_output <- umap$fit_transform(as.matrix(x = PCA))
  rownames(umap_output)=rownames(PCA)
  colnames(umap_output)=paste0("UMAP",1:max_dim)
  
  return(umap_output)
}

p2wrapper <- function(counts,n_neighbors=30,min_dist=0.3,k=100,npcs=200,selpc=T,...) {
  rownames(counts) <- make.unique(rownames(counts))
  p2 <- Pagoda2$new(counts,log.scale=FALSE,n.cores=parallel::detectCores()/2,...)
  p2$adjustVariance(plot=T,gam.k=10)
  p2$calculatePcaReduction(nPcs=npcs,n.odgenes=NULL,maxit=1000)
  if (selpc){
    x <- cbind(1:npcs, p2$misc$PCA$d)
    line <- x[c(1, nrow(x)),]
    proj <- princurve::project_to_curve(x, line)
    optpoint <- which.max(proj$dist_ind)-1
    dev.new(width=5, height=4)
    par(mfrow=c(1,1))
    plot(x,xlab="PC", ylab="Variance explained")
    abline(v=optpoint,lty=2,col=2)
    cat(paste0(optpoint," PCs retained\n"))
    npcs=optpoint
    p2$calculatePcaReduction(use.odgenes = T, name='PCA', 
                             nPcs=optpoint, maxit=1000)
  }
  
  p2$makeKnnGraph(k=k,type='PCA',center=T,distance='cosine');
  p2$getKnnClusters(method=conos::leiden.community,type='PCA',name = "leiden",resolution=.5)
  
  # Produce UMAP embedding
  cat("Computing UMAP... ")
  p2$embeddings$PCA$UMAP=doUMAP(PCA = p2$reductions$PCA[,1:npcs],n_neighbors = n_neighbors,min_dist = min_dist)
  cat("done\n")
  invisible(p2)
}


p2webwrapper <- function(p2,clus=p2$clusters$PCA$leiden, app.title = "Pagoda2", extraWebMetadata = NULL, 
    n.cores = 4) 
{
    cat("Calculating hdea...\n")
    hdea <- p2$getHierarchicalDiffExpressionAspects(type = "PCA", 
        clusterName = "leiden", z.threshold = 3, n.cores = n.cores)
    metadata.forweb <- list()
    metadata.forweb$leiden <- p2.metadata.from.factor(clus, 
        displayname = "Leiden")
    metadata.forweb <- c(metadata.forweb, extraWebMetadata)
    genesets <- hierDiffToGenesets(hdea)
    appmetadata = list(apptitle = app.title)
    cat("Making KNN graph...\n")
    p2$makeGeneKnnGraph(n.cores = n.cores)
    make.p2.app(p2, additionalMetadata = metadata.forweb, geneSets = genesets, 
        dendrogramCellGroups = clus, show.clusters = F, 
        appmetadata = appmetadata)
}

# Load the files

In [ ]:
#cd <- t.load.10x.data(list(E11_CTRL='_Data/ML5_CTRL/',E11_MUT="_Data/ML4_MUT/"))
cd <- t.load.10x.data(list(ML4='/home/lfaure/backup/ML4/',ML5='/home/lfaure/backup/ML5/'))

#cd <- t.load.10x.data(list(ML9='_Data/ML9_E11-2/',ML10='_Data/ML10_E10/'))


# Filter cells

In [ ]:
counts <- gene.vs.molecule.cell.filter(cd[[1]],max.cell.size = 1e5)
for (i in 2:length(cd)){
  counts <-cbind(counts,gene.vs.molecule.cell.filter(cd[[i]],max.cell.size = 1e5))
}

counts <- counts[rowSums(counts)>=10,]
mito.genes <- grep(pattern = "^mt-", x = rownames(x = counts), value = TRUE)
percent.mito <- Matrix::colSums(counts[mito.genes, ])/Matrix::colSums(counts)
counts = counts[,percent.mito<.1]


# Make pagoda2 object

## PCA and elbow curve selection to retain a smaller number of PCs

In [ ]:
p2=p2wrapper(counts,k=100,npcs = 50,selpc = F,batch=batch)

batch=factor(sapply(strsplit(rownames(p2$counts),"_"),"[[",1))
names(batch)=rownames(p2$counts)

p2$plotEmbedding(type="PCA",embeddingType = "UMAP",groups=batch)
write.table(p2$embeddings$PCA$UMAP,"_Output/UMAP.csv")
saveRDS(p2,"_Output/p2_ML4_5.rds")


In [ ]:
pl=ggplot(data.frame(p2$embeddings$PCA$UMAP,leiden=p2$clusters$PCA$leiden))+
    geom_point(aes(x=UMAP1,y=UMAP2),size=1.5)+geom_point(aes(x=UMAP1,y=UMAP2,col=leiden),size=1)+
  scale_color_stata("s1color")+
  theme_void()+theme(aspect.ratio = 1,legend.direction = "horizontal",legend.position = c(.2,.1))

ggsave("_Figures/ML4-5_leiden.png",pl,width = 8,height = 8,dpi = 300)

pl=ggplot(data.frame(p2$embeddings$PCA$UMAP,leiden=p2$clusters$PCA$leiden))+
    geom_point(aes(x=UMAP1,y=UMAP2),size=1.5)+geom_point(aes(x=UMAP1,y=UMAP2,col=batch),size=1)+
  theme_void()+theme(aspect.ratio = 1,legend.direction = "horizontal",legend.position = c(.2,.1))

ggsave("_Figures/ML4-5_batch.png",pl,width = 8,height = 8,dpi = 300)

In [ ]:
res=p2$getDifferentialGenes("PCA","leiden");
resf=lapply(res,function(x) x[x$M>1 & x$fe>.5 & x$highest,]);

dendro=generateDendrogramOfGroups(p2,p2$clusters$PCA$leiden)
dend=as.dendrogram(dendro$hc)

resf_s=lapply(resf,function(x) x[order(x$M,decreasing = T),])[dendro$hc$order]


s=CreateSeuratObject(t(p2$counts));s@data=s@raw.data

s@meta.data$leiden=p2$clusters$PCA$leiden
topgenes=do.call(c,lapply(resf_s,function(x) rownames(x)[1:4]))
topgenes=unique(topgenes[!is.na(topgenes)])
pl=DotPlot(s,genes.plot = topgenes,group.by = "leiden",plot.legend = T,do.return = T)+theme_pubr()+ theme(axis.text.x = element_text(angle = 90,vjust=0.5, hjust = 1))+
  scale_y_discrete(limits=as.character(dendro$hc$order))+
  theme(axis.title.y=element_blank(),axis.title.x=element_blank(),
        plot.margin = unit(c(0,5.5,5.5,0), "pt"))

pa3=ggplot(as.ggdend(dend),horiz = T,labels = F)+theme_void()+
  theme(plot.margin = unit(c(-40,-40,-40,-40), "pt"))

combi=plot_grid(pa3,pl,align = "hv",nrow = 1,axis = "tb",rel_widths = c(1,5))

ggsave("_Figures/ML4-5_Dotplot.png",combi,width = 12,height = 8,dpi=300)


cellcounts=sapply(levels(p2$clusters$PCA$leiden),function(x) table(factor(sapply(strsplit(names(p2$clusters$PCA$leiden[p2$clusters$PCA$leiden%in%x]),"_"),"[[",1))))
prop=apply(cellcounts/c(table(batch)),2,function(x) x/sum(x))

clu=as.numeric(p2$clusters$PCA$leiden)
for (i in 1:nlevels(p2$clusters$PCA$leiden)){clu[clu%in%i]=prop[1,i]}
names(clu)=names(p2$clusters$PCA$leiden)

compa=data.frame(p2$embeddings$PCA$UMAP,prop=clu)

pl5=ggplot(compa)+geom_point(aes(x=UMAP1,y=UMAP2,col=prop))+theme_bw()+
    scale_color_distiller(palette = "RdBu", limits = c(0.28,.72))+theme_void()+
  theme(aspect.ratio = 1,legend.direction = "horizontal",legend.position = c(.2,.1))

ggsave("_Figures/ML4_5_compo.png",pl5,width = 8,height = 8,dpi = 300)



mart <- useMart(biomart = "ensembl", dataset = "mmusculus_gene_ensembl")

wb <- createWorkbook(type="xlsx")
k=1;pl=list();genestostring_down=list();genestostring_up=list()
for (cl in c(2,4,1,3)){
  selclu=p2_epcam$clusters$PCA$leiden2%in%cl
  clunames=names(p2_epcam$clusters$PCA$leiden2)[selclu]
  subbatch=sapply(strsplit(clunames,"_"),"[",1)
  names(subbatch)=clunames
  res=p2_epcam$getDifferentialGenes(groups=subbatch)
  res=res$ML4
  
  sheet <- createSheet(wb, sheetName = paste0("leiden ",cl))
  xlsx.addTable(wb,sheet, res,startCol = 1,row.names = T,startRow = 1)
  
  res$genes=rownames(res)
  res$chr=NA
  chr=getBM(attributes = c("external_gene_name","chromosome_name"), 
            filters = "external_gene_name", values = res$genes, 
            bmHeader = T, mart = mart)
  res[chr$`Gene name`,]$chr=chr$`Chromosome/scaffold name`
  idxtorem=grep("Rp[s|l]|mt|Hmg|Hba|Gm|Cox|Hbb",
                rownames(res),value = F)
  res$genes[idxtorem]=""
  res$genes[res$chr%in%"Y"]=""
  res$genes[abs(res$M)<.3]=""
  
  genestostring_down[[k]]=res[res$M<0,]$genes[!res[res$M<0,]$genes%in%""]
  genestostring_up[[k]]=res[res$M>0,]$genes[!res[res$M>0,]$genes%in%""]
  
  
  volc=data.frame(pval=2*pnorm(-abs(res$Z)),folchange=res$M,genes=(res$genes));
  pl[[k]]=ggplot(volc,aes(x=folchange,y=pval,label=genes))+geom_point(color="grey")+
    geom_point(data=subset(volc,abs(folchange)>.3),color="red")+
    scale_y_log10()+theme_pubr()+geom_vline(xintercept = 0)+
    geom_text_repel(segment.size = .2,segment.color = "grey50",force = 10)+ggtitle(cl)
  
  # res=res[abs(res$M)>.15,]
  # average=data.frame(ML6=colMeans(p2_epcam$counts[clunames[subbatch%in%"ML6"],]),
  #                    ML7=colMeans(p2_ML6_7$counts[clunames[subbatch%in%"ML7"],]))
  # #average=log1p(average)
  # average$genes=""
  # idxtorem=grep("Rp[s|l]|mt|Hmg|Hba|Gm|Cox|Hbb",
  #               rownames(res),value = F)
  # average[rownames(res)[-idxtorem],]$genes=rownames(res)[-idxtorem]
  # 
  # print(length(rownames(res)[-idxtorem]))
  # pl[[k]]=(ggplot(average,aes(x=ML6,y=ML7,label=genes))+
  #   geom_point(color = ifelse(average$genes == "", "grey50", "red"))+theme_pubr()+
  #   geom_text_repel()+ggtitle(cl))
  k=k+1
}

saveWorkbook(wb, "ML4-5_epcam_tables.xlsx")

ggsave("epcam_Ml4-5_final2.png",do.call("grid.arrange", c(pl, ncol=2,nrow=2)),width=16,height = 16,dpi=300)

In [ ]:
library(r2excel)
library(ggrepel)
library(ggpubr)
library(gridExtra)
mart <- useMart(biomart = "ensembl", dataset = "mmusculus_gene_ensembl")
wb <- createWorkbook(type="xlsx")
k=1;pl=list();genestostring_down=list();genestostring_up=list()
for (cl in 1:12){
  selclu=p2$clusters$PCA$leiden%in%cl
  clunames=names(p2$clusters$PCA$leiden)[selclu]
  subbatch=sapply(strsplit(clunames,"_"),"[",1)
  names(subbatch)=clunames
  res=p2$getDifferentialGenes(groups=subbatch)
  res=res$ML4
  
  sheet <- createSheet(wb, sheetName = paste0("leiden ",cl))
  xlsx.addTable(wb,sheet, res,startCol = 1,row.names = T,startRow = 1)
  
  res$genes=rownames(res)
  res$chr=NA
  chr=getBM(attributes = c("external_gene_name","chromosome_name"), 
            filters = "external_gene_name", values = res$genes, 
            bmHeader = T, mart = mart)
  res[chr$`Gene name`,]$chr=chr$`Chromosome/scaffold name`
  idxtorem=grep("Rp[s|l]|mt|Hmg|Hba|Gm|Cox|Hbb",
                rownames(res),value = F)
  res$genes[idxtorem]=""
  res$genes[res$chr%in%"Y"]=""
  res$genes[abs(res$M)<.3]=""
  
  genestostring_down[[k]]=res[res$M<0,]$genes[!res[res$M<0,]$genes%in%""]
  genestostring_up[[k]]=res[res$M>0,]$genes[!res[res$M>0,]$genes%in%""]
  
  volc=data.frame(pval=2*pnorm(-abs(res$Z)),folchange=res$M,genes=(res$genes));
  pl[[k]]=ggplot(volc,aes(x=folchange,y=pval,label=genes))+geom_point(color="grey")+
    geom_point(data=subset(volc,abs(folchange)>.3),color="red")+
    scale_y_log10()+theme_pubr()+geom_vline(xintercept = 0)+
    geom_text_repel(segment.size = .2,segment.color = "grey50",force = 10)+ggtitle(cl)
  
  k=k+1
}

saveWorkbook(wb, "ML4-5_tables.xlsx")

ggsave("ML4-5_volcplots.png",do.call("grid.arrange", c(pl, ncol=5,nrow=3)),width=30,height = 18,dpi=300)

ggsave("ML4-5_final.png",do.call("grid.arrange", c(pl[c(3,7,10)], ncol=3,nrow=1)),width=24,height = 8,dpi=300)



In [ ]:
library(SingleCellExperiment)
library(scds)

sce <- SingleCellExperiment(assays = list(counts = t(p2w$originalP2object$misc$rawCounts)))

sce = cxds_bcds_hybrid(sce,versio)

In [ ]:

res=p2$getDifferentialGenes(groups = p2$clusters$PCA$leiden,upregulated.only = T)

p2_epcam=p2wrapper(counts[,p2$clusters$PCA$leiden%in%which(sapply(res,function(x) any(rownames(x)%in%"Epcam")))],k = 100,npcs = 50,selpc = F,batch=batch)

batch=factor(sapply(strsplit(rownames(p2_epcam$counts),"_"),"[[",1))
names(batch)=rownames(p2_epcam$counts)

p2_epcam$plotEmbedding(type="PCA",embeddingType = "UMAP",groups=batch)

res=p2_epcam$getDifferentialGenes(groups = batch,upregulated.only = T)

percent.mito = percent.mito[rownames(p2_epcam$counts)]

library(Seurat)
s=CreateSeuratObject(t(as.matrix(p2_epcam$counts)))
s=NormalizeData(s)
s@meta.data$percent.mito=percent.mito
s@data=as.matrix(t(p2_epcam$counts))
s=ScaleData(s,vars.to.regress = "percent.mito",num.cores = 4,do.par = T)
p2_epcam$reductions$regressed=t(s@scale.data)
p2_epcam$calculatePcaReduction(type = "regressed",n.odgenes=NULL,nPcs = 50,maxit = 1000);
p2_epcam$embeddings$PCA$UMAP=doUMAP(PCA = p2_epcam$reductions$PCA,n_neighbors = 30,min_dist = .3)

load("~/Projects/Face/_Output/p2w_ML8-9.RData")
library(conos)
p2_ML8_9=p2w$originalP2object
rm(p2w);gc()
con <- Conos$new(list(ML4_5=p2_epcam,ML8_0=p2_ML8_9), n.cores=4)

con$buildGraph()

con$plotPanel(groups = p2_ML8_9$clusters$PCA$leiden2,embedding="UMAP")

new.label.info <- con$propagateLabels(labels = p2_ML8_9$clusters$PCA$leiden2, verbose=T )

con$plotPanel(colors=new.label.info$uncertainty, show.legend=T, legend.title="Uncertainty", legend.pos=c(1, 0),embedding="UMAP")

con$plotPanel(groups=new.label.info$labels, show.legend=F,embedding = "UMAP")

p2_epcam$clusters$PCA$leiden2=factor(new.label.info$labels[rownames(p2_epcam$counts)])

saveRDS(p2_epcam,"_Output/p2_ML6-7_epcam.rds")

pl4=ggplot(data.frame(p2_epcam$embeddings$PCA$UMAP,leiden=p2_epcam$clusters$PCA$leiden2))+
    geom_point(aes(x=UMAP1,y=UMAP2),size=1.5)+geom_point(aes(x=UMAP1,y=UMAP2,col=leiden),size=1)+
  scale_color_manual(values=tableau_color_pal("Tableau 20")(14)[as.numeric(levels(factor(p2_epcam$clusters$PCA$leiden2)))])+
  theme_void()+theme(aspect.ratio = 1,legend.direction = "horizontal",legend.position = c(.3,.9))
ggsave("_Figures/ML4_5_epcam_leiden.png",pl4,width = 4,height = 4,dpi = 300)


pl5=ggplot(data.frame(p2_epcam$embeddings$PCA$UMAP,batch=batch))+
    geom_point(aes(x=UMAP1,y=UMAP2,col=batch),size=1,alpha=.7)+
  theme_void()+theme(aspect.ratio = 1,legend.direction = "horizontal",legend.position = c(.3,.9))
ggsave("_Figures/ML4_5_epcam_batches.png",pl5,width = 6,height = 6,dpi = 300)

cellcounts=sapply(levels(p2_epcam$clusters$PCA$leiden2),function(x) table(factor(sapply(strsplit(names(p2_epcam$clusters$PCA$leiden2[p2_epcam$clusters$PCA$leiden2%in%x]),"_"),"[[",1))))
prop=apply(cellcounts/c(table(batch)),2,function(x) x/sum(x))

clu=as.numeric(p2_epcam$clusters$PCA$leiden2)
for (i in levels(p2_epcam$clusters$PCA$leiden2)){clu[p2_epcam$clusters$PCA$leiden2%in%i]=prop[1,as.character(i)]}
names(clu)=names(p2_epcam$clusters$PCA$leiden2)

compa=data.frame(p2_epcam$embeddings$PCA$UMAP,prop=clu)

pl6=ggplot(compa)+geom_point(aes(x=UMAP1,y=UMAP2,col=prop))+theme_bw()+
    scale_color_distiller(palette = "RdBu", limits = c(0.25,.75))+theme_void()+
  theme(aspect.ratio = 1,legend.direction = "horizontal",legend.position = c(.3,.9))

ggsave("_Figures/ML4_5_epcam_compo.png",pl6,width = 6,height = 6,dpi = 300)


## make pagoda2 web app

In [ ]:
library(conos)

con <- Conos$new(list(ML4_5=`p2_ML4-5_epcam`,ML8_9=p2w$originalP2object), n.cores=4)

con$buildGraph()

con$plotPanel(groups = p2_ML10$clusters$PCA$leiden2,embedding="UMAP")

new.label.info <- con$propagateLabels(labels = p2w$originalP2object$clusters$PCA$leiden2, verbose=T )

con$plotPanel(colors=new.label.info$uncertainty, show.legend=T, legend.title="Uncertainty", legend.pos=c(1, 0),embedding="UMAP")

con$plotPanel(groups=new.label.info$labels, show.legend=F,embedding = "UMAP")

`p2_ML4-5_epcam`$clusters$PCA$leiden=factor(new.label.info$labels[rownames(`p2_ML4-5_epcam`$counts)])

# generate go environmet for GO terms enrichement analyis
p2$n.cores=3
go.env <- p2.generate.mouse.go(p2)
p2$testPathwayOverdispersion(setenv = go.env,
                             recalculate.pca=F,
                             correlation.distance.threshold = 0.95)
myGeneNames <- colnames(p2$counts)
goSets <- p2.generate.mouse.go.web(myGeneNames)
deSets <- get.de.geneset(p2, groups = p2$clusters$PCA$leiden, prefix = 'de_')
geneSets <- c(goSets, deSets)

# Prepare metadata to show on web app
library(ggthemes)
additionalMetadata <- list()

additionalMetadata$leiden <- p2.metadata.from.factor(p2$clusters$PCA$leiden, displayname = 'Leiden', s = 0.7, v = 0.8,start = 0, end = 0.5,pal = ggthemes::tableau_color_pal("Tableau 20")(20)[as.numeric(levels(p2$clusters$PCA$leiden))])

p2w <- make.p2.app(
  p2,
  dendrogramCellGroups = p2$clusters$PCA$leiden,
  additionalMetadata = additionalMetadata,
  geneSets = geneSets,
  show.clusters = FALSE # Hide the clusters that were used for the dendrogram from the metadata
)
p2w$serializeToStaticFast("_Output/p2w_ML4-5_epcam.bin")
save(p2w,file="_Output/p2w_ML4-5_epcam.RData")

# launch web app
show.app(p2w,"p2")

In [ ]:
p2_mesenchyme=p2wrapper(counts[,p2$clusters$PCA$leiden%in%1:6],k = 100,npcs = 30,selpc = F,batch=batch)

s=CreateSeuratObject(t(p2_mesenchyme$counts));s@data=s@raw.data

dendro=generateDendrogramOfGroups(p2_mesenchyme,p2_mesenchyme$clusters$PCA$leiden)
dend=as.dendrogram(dendro$hc)

res=p2_mesenchyme$getDifferentialGenes("PCA","leiden");
resf=lapply(res,function(x) x[x$M>.8 & x$fe>.5 & x$highest,]);
resf_s=lapply(resf,function(x) x[order(x$M,decreasing = T),])[dendro$hc$order]
s@ident=p2_mesenchyme$clusters$PCA$leiden
selgenes=unique(do.call(c,lapply(resf_s,function(x) rownames(x)[1:4])))
selgenes=selgenes[!is.na(selgenes)]
pl=DotPlot(s,genes.plot = selgenes,group.by = "leiden",plot.legend = T,do.return = T)+theme_pubr()+ theme(axis.text.x = element_text(angle = 90,vjust=0.5, hjust = 1))+
  scale_y_discrete(limits=as.character(dendro$hc$order))+
  theme(axis.title.y=element_blank(),axis.title.x=element_blank(),
        plot.margin = unit(c(0,5.5,5.5,0), "pt"))

pa3=ggplot(as.ggdend(dend),horiz = T,labels = F)+theme_void()+
  theme(plot.margin = unit(c(-40,-40,-40,-40), "pt"))

combi=plot_grid(pa3,pl,align = "hv",nrow = 1,axis = "tb",rel_widths = c(1,5))

ggsave("_Figures/ML4-5_mesenchymes_Dotplot.png",combi,width = 10,height = 6,dpi=300)

saveRDS(p2_mesenchyme,"_Output/p2_ML4-5_mesenchymes.rds")


p2_mesenchyme$n.cores=3
go.env <- p2.generate.mouse.go(p2_mesenchyme)
p2_mesenchyme$testPathwayOverdispersion(setenv = go.env,
                             recalculate.pca=F,
                             correlation.distance.threshold = 0.95)
myGeneNames <- colnames(p2_mesenchyme$counts)
goSets <- p2.generate.mouse.go.web(myGeneNames)
deSets <- get.de.geneset(p2_mesenchyme, groups = p2_mesenchyme$clusters$PCA$leiden, prefix = 'de_')
geneSets <- c(goSets, deSets)


## new clustering using TFs

In [ ]:
expr=p2$counts[,p2$misc$odgenes]

N_perm <- 20
expl_var_perm <- matrix(NA, ncol = 100, nrow = N_perm)

perm=parallel::mclapply(1:N_perm, function(k){
  expr_perm <- apply(expr,2,sample)
  PC_perm <- irlba::prcomp_irlba(expr_perm,n = 100, center=TRUE, scale=FALSE)
  return(PC_perm$sdev^2/sum(PC_perm$sdev^2))
},mc.cores = 20)




tocheck=rep(1,dim(p2$counts)[1])
names(tocheck)=rownames(p2$counts)
tocheck[strsplit(readLines("_K/sub1"),",")[[1]]]=2
tocheck[strsplit(readLines("_K/sub2"),",")[[1]]]=3
tocheck[strsplit(readLines("_K/sub3"),",")[[1]]]=4
tocheck[strsplit(readLines("_K/sub4"),",")[[1]]]=5
res=p2$getDifferentialGenes(groups=tocheck)


tokeep=!tocheck>1 & !p2$clusters$PCA$leiden%in%c(7,8,12)
cc_F=p2w$originalP2object$misc$pathwayOD$xv[3,tokeep]

p2=p2wrapper(counts[,tokeep])

s=CreateSeuratObject(t(p2$misc$rawCounts))
#s@meta.data$cc.aspect=p2w$originalP2object$misc$pathwayOD$xv[3,]


s@data=t(p2w$originalP2object$counts)
#s=ScaleData(s,vars.to.regress = c("cc.aspect","dnarep.aspect"))

s=ScaleData(s,vars.to.regress = c("cc.aspect"),do.par = T,num.cores = 2)



s=FindVariableGenes(s)
s=RunPCA(s,pcs.compute = 200)

x <- cbind(1:200,s@dr$pca@sdev)
line <- x[c(1, nrow(x)),]
proj <- princurve::project_to_curve(x, line)
optpoint <- which.max(proj$dist_ind)-1



u=doUMAP(s@dr$pca@cell.embeddings[,1:optpoint],n_neighbors = 30,min_dist = .3)

p2$embeddings$PCA.corr$UMAP=u

p2$reductions$PCA.corr=GetCellEmbeddings(s,"pca")[,1:optpoint]
p2$makeKnnGraph(k=100,type='PCA.corr',center=T,distance='cosine');
# using leiden community detection, taken from conos package
p2$getKnnClusters(method=conos::leiden.community,type='PCA.corr',
                  name = "leiden",resolution=.5)





# GO term "transcription regulator activity" GO:0140110
GOTF=intersect(unique(unlist(read.table("_prior/GO_TF_0140110.txt",stringsAsFactors = F))),colnames(p2$counts))

p2red=p2wrapper(counts[unique(c(GOTF)),],n_neighbors = 150,min_dist = .5)

p2$embeddings$PCA.red$UMAP=p2red$embeddings$PCA$UMAP
p2$clusters$PCA.red$leiden=p2red$clusters$PCA$leiden
p2$reductions$PCA.red=p2red$reductions$PCA

additionalMetadata <- list()

additionalMetadata$leiden <- p2.metadata.from.factor(p2$clusters$PCA$leiden, displayname = 'Leiden', s = 0.7, v = 0.8,start = 0, end = 0.5,pal = tableau_color_pal(palette = "Tableau 20")(nlevels(p2$clusters$PCA$leiden)))

additionalMetadata$leiden.corr <- p2.metadata.from.factor(p2$clusters$PCA.corr$leiden, displayname = 'Leiden', s = 0.7, v = 0.8,start = 0, end = 0.5,pal = tableau_color_pal(palette = "Tableau 10")(nlevels(p2$clusters$PCA.corr$leiden)))

additionalMetadata$time=p2w$cellmetadata$time

p2w <- make.p2.app(
  p2,
  dendrogramCellGroups = p2$clusters$PCA.corr$leiden,
  additionalMetadata = additionalMetadata,
  geneSets = p2w$geneSets,
  show.clusters = FALSE # Hide the clusters that were used for the dendrogram from the metadata
)

saveRDS(p2w,"_Output/p2w_ML8-11_corr.rds")

## Some plots

In [ ]:
clus.corr=p2$clusters$PCA.corr$leiden
clus.corr=plyr::mapvalues(clus.corr,levels(clus.corr),
                          paste0(levels(clus.corr),"_corr"))
clus.red=p2$clusters$PCA.red$leiden
clus.red=plyr::mapvalues(clus.red,levels(clus.red),
                          paste0(levels(clus.red),"_red"))

compa=data.frame(name=c(levels(clus.corr),levels(clus.red)))

links=data.frame(source=NA,target=NA,value=NA)

k=1
for (lev in 1:nlevels(clus.corr)){
  for (lev2 in 1:nlevels(clus.red)){
    links[k,]$source=lev-1
    links[k,]$target=nlevels(clus.corr)+lev2-1
    links[k,]$value=length(intersect(names(clus.corr[clus.corr%in%levels(clus.corr)[lev]]),
              names(clus.red[clus.red%in%levels(clus.red)[lev2]])))
    k=k+1
  }
}

compa=list(compa,links)
names(compa)=c("nodes","links")

simi=sapply(unique(compa$links$source),
            function(x) which.max(compa$links$value[compa$links$source==x]))

dup=which(simi==simi[duplicated(simi)])


emb=data.frame(p2$embeddings$PCA.corr$UMAP)
emb$clusters.corr=p2$clusters$PCA.corr$leiden
umap.clus=ggplot(emb)+geom_point(aes(x=UMAP1,y=UMAP2),size=1.5)+
  geom_point(aes(x=UMAP1,y=UMAP2,col=clusters.corr),size=1)+
  scale_color_tableau(name="OD genes clusters","Tableau 10")+theme_void()+
  theme(aspect.ratio = 1,legend.position = "bottom")

ggsave("_Figures/ML1_umap.clusters.corr.png",umap.clus,width = 8,height = 8)

emb$clusters.red=p2$clusters$PCA.red$leiden

umap.clus=ggplot(emb)+geom_point(aes(x=UMAP1,y=UMAP2),size=1.5)+
  geom_point(aes(x=UMAP1,y=UMAP2,col=clusters.red),size=1)+
  scale_color_manual(name="TF clusters",
                     values=c("#F28E2B",  "#FF9DA7","#9C755F","#BAB0AC",
                              "#76B7B2","#E15759","#EDC948","#59A14F","#B07AA1"))+
  theme_void()+
  theme(aspect.ratio = 1,legend.position = "bottom")

ggsave("_Figures/ML1_umap.clusters.red.png",umap.clus,width = 8,height = 8)

my_color <- 'd3.scaleOrdinal() .domain(["1_corr", "2_corr","3_corr", "4_corr", "5_corr", "6_corr", "7_corr", "1_red", "2_red", "3_red", "4_red", "5_red","6_red","7_red","8_red","9_red"]) .range(["#4E79A7", "#F28E2B" , "#E15759", "#76B7B2", "#59A14F", "#EDC948" , "#B07AA1", "#F28E2B",  "#FF9DA7","#9C755F","#BAB0AC","#76B7B2","#E15759","#EDC948","#59A14F","#B07AA1"])'

sN=sankeyNetwork(Links = compa$links, Nodes = compa$nodes, Source = "source",
              Target = "target", Value = "value", NodeID = "name",
              units = "Cells", fontSize = 18, nodeWidth = 50,colourScale = my_color)

capture_widget(sN, path="_Figures/ML1_Sankey.png",height=1000, width=1000)